### Import Modules

In [ ]:
import pandas as pd

In [ ]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error

In [ ]:
import mlflow

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pickle

### Set Experiment and Tracking URI

In [ ]:
mlflow.set_tracking_uri("sqlite:///../mlruns.db")

In [ ]:
mlflow.set_experiment("NYC-Duration-Exp4")

### Read and Preprocess data

In [ ]:
def read_data(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()
    
    return df

In [ ]:
train_data_path = '../data/yellow_tripdata_2023-01.parquet'
val_data_path = '../data/yellow_tripdata_2023-02.parquet'

In [ ]:
df_train = read_data(train_data_path)
df_val = read_data(val_data_path)

In [ ]:
def compute_features(df):
    str_columns = ["PULocationID", "DOLocationID"]
    df[str_columns] = df[str_columns].astype('str')
    df['PU_DO'] = df_train["PULocationID"] + "_" + df_train["DOLocationID"]

    return df

def extract_features_target(df):
    df = compute_features(df)
    
    categorical = ['PU_DO']
    numerical = ['trip_distance']

    return df[categorical + numerical], df.duration.values

In [ ]:
X_train, y_train = extract_features_target(df_train)
X_val, y_val = extract_features_target(df_val)

In [ ]:
## Fit-Transform vectorizer

# get columns as dict
train_dicts = X_train.to_dict(orient='records')
val_dicts = X_val.to_dict(orient='records')

# fit transform dict vectorizer
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
X_val = dv.transform(val_dicts)

In [ ]:
X_train.shape, X_val.shape

In [ ]:
y_train = df_train.duration.values
y_val = df_val.duration.values

In [ ]:
y_train.shape, y_val.shape

### Train and evaluate

In [ ]:
# Define model
alpha = 0.5
lr = Ridge(alpha)

# Train model
lr.fit(X_train, y_train)

# Eval model
y_train_pred = lr.predict(X_train)
train_rmse = mean_squared_error(y_train, y_train_pred, squared=False)

y_val_pred = lr.predict(X_val)
val_rmse = mean_squared_error(y_val, y_val_pred, squared=False)

In [ ]:
print(f"Train RMSE: {train_rmse:.2f}")
print(f"Validation RMSE: {val_rmse:.2f}")

### Tracking with MLFlow

In [ ]:
mlflow.autolog(disable=True)

In [ ]:
with mlflow.start_run():
    # Log tag
    mlflow.set_tag("model type", "ridge")
    
    # Log params
    mlflow.log_param("train dp", train_data_path)
    mlflow.log_param("val dp", val_data_path)
    mlflow.log_param("alpha", alpha)

    # Log metrics
    mlflow.log_metric("Train RMSE", train_rmse)
    mlflow.log_metric("Val RMSE", val_rmse)

### Save model and vectorizers as artifact

In [ ]:
# Save both model and vectorizers with pickle
# ...

In [ ]:
# Log params, metrics, and artifact
# ...

### Save vectorizer only as artifact

In [ ]:
from mlflow.models import infer_signature

In [ ]:
# Save vectorizer with pickle
# ...

In [ ]:
# Log params, metrics, artifact and model
# ...
# Use signature when logging model

### Load model

#### Case: model and vectorizer logged as artifact 

In [ ]:
run_id = "d3d954b0d92540b5984a22ddbf010a9a"
artifact_name = "lin_reg.bin"

In [ ]:
artifact_path = mlflow.artifacts.download_artifacts(artifact_uri=f"runs:/{run_id}/{artifact_name}")

In [ ]:
artifact_path

In [ ]:
with open(artifact_path, 'rb') as f_in:
    loaded_dv, loaded_lr = pickle.load(f_in)

In [ ]:
loaded_dv, loaded_lr

#### Case: only vectorizer as artifact

In [ ]:
# run_id = ""
# artifact_name = ""

In [ ]:
# Build artifact uri runs:/{run_id}/{artifact_path}
artifact_path = mlflow.artifacts.download_artifacts(artifact_uri=f"runs:/{run_id}/{artifact_name}")

In [ ]:
artifact_path

In [ ]:
# load vectorizer
# ...

In [ ]:
model_uri = f"runs:/{run_id}/models"
model_uri

In [ ]:
# Load model
# ...

In [ ]:
# Use model to predict
# ...